In [1]:
from gz_import import *
from gz_mainlib import *
from gz_plotlib import *
from gz_dwnllib import *
from gz_rwlib import *
from gz_const import *

mainver()
plotver()
rwver()
dwnlver()

Main ver: 67
Plot ver: 67
RW ver: 67
Download ver: 67


In [3]:
#self._base_url = 'https://query1.finance.yahoo.com'
#data = self.session.get(url=url, params=params, proxies=proxy)

In [3]:
base_url = 'https://query1.finance.yahoo.com'

In [4]:
url = "{}/v8/finance/chart/{}".format(base_url, 'AAPL')

In [2]:
def download_ticker_info(tic):
    scrape_url = 'https://finance.yahoo.com/quote'
    ticker_url = '{}/{}'.format(scrape_url, tic)
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) \
            Chrome/39.0.2171.95 Safari/537.36'}
    
    html = requests.get(ticker_url, headers=headers).text
    
    
    #несколько попыток
    if 'QuoteSummaryStore' not in html:
        html = session.get(ticker_url).text
        if 'QuoteSummaryStore' not in html:
            html = session.get(ticker_url).text
            if 'QuoteSummaryStore' not in html:
                return {}
    
    json_str = html.split('root.App.main =')[1].split('(this)')[0].split(';\n}')[0].strip()
    data = json.loads(json_str)['context']['dispatcher']['stores']['QuoteSummaryStore']
    
    if 'summaryProfile' not in html:
        return {}
    
    ticprofile = data['summaryProfile']
    
    return ticprofile


In [3]:
download_ticker_info('AAPL')

{'zip': '95014',
 'sector': 'Technology',
 'fullTimeEmployees': 100000,
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch, and other Apple-branded and third-party accessories. It also provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store, that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Music, which offers users a curated listening experience with on-demand r

In [10]:
def download_tinfo_add_todict(tic, yfd, que):
    #stock = yf.Ticker(tic)
    yfinfo =  download_ticker_info(tic)
    yfinfo.update({'ticker':tic})
    yfd.update({tic:yfinfo})
    
    if yfinfo == {}:
        que.put(tic+' void info.')
    else:
        que.put(tic+' info ok.')
    
def load_mt_yahoo_ticsinfo(tics, pool_size=75):
    yf_list = []
    yf_dict = {}
    #один тикер
    if type(tics) is str:
        tics = [tics]
    que = Queue()
    error_list = []
    t_print = Mythread(print_mt, (que,))  
    t_print.start()
    t_pool = Threadpool(pool_size)

    for tic in tics:
        if t_pool.isfull():
            t_pool.start_and_wait()
        t = Mythread(download_tinfo_add_todict, 
                     (tic, yf_dict, que), tic)
        t_pool.addthread(t)
    
    #если остались невыполненные работы дорабатываем        
    if t_pool.poolsize > 0: 
        t_pool.start_and_wait()
    que.put(100) #сигнал о завершении
    sleep(0.1)
    return yf_dict #pd.DataFrame(yf_list)

In [11]:
df_instruments = pd.read_hdf('dat.h5', 'df_instruments')
all_us_ticks = sorted(df_instruments[df_instruments['currency'] == 'USD']['ticker'].tolist())

In [9]:
d_tics_info = load_mt_yahoo_ticsinfo(all_us_ticks, pool_size=200)

ZTS info ok..ok..
Print thread stopped


In [12]:
d_tics_info2 = load_mt_yahoo_ticsinfo(['TSLA', 'SPCE'], pool_size=200)

SPCE info ok.
Print thread stopped


In [13]:
d_tics_info2

{'TSLA': {'zip': '94304',
  'sector': 'Consumer Cyclical',
  'fullTimeEmployees': 70757,
  'longBusinessSummary': 'Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, and internationally. The company operates in two segments, Automotive, and Energy Generation and Storage. The Automotive segment offers electric vehicles, as well as sells automotive regulatory credits. It provides sedans and sport utility vehicles through direct and used vehicle sales, a network of Tesla Superchargers, and in-app upgrades; and purchase financing and leasing services. This segment is also involved in the provision of non-warranty after-sales vehicle services, sale of used vehicles, retail merchandise, and vehicle insurance, as well as sale of products through its subsidiaries to third party customers; services for electric vehicles through its company-owned service locations, and Tesla mobile service techn

In [11]:
for x,y in d_tics_info.items():
    if x == {}:
        print('viod')

In [75]:
hdf5_writejson('dat.h5', 'yfinfo', d_tics_info)

'/yfinfo'

In [6]:
hdf5_contain('dat.h5')
#yfinfo = hdf5_readjson('dat.h5', 'yfinfo')
#hdf5_delete_dataset('dat.h5', 'yfinfo')

['df_brent',
 'df_instruments',
 'dfm',
 'dfm_coint',
 'dfm_corr',
 'dfm_doclh',
 'dfy',
 'dfy_coint',
 'dfy_corr',
 'dfy_doclh',
 'l_industry_retrisksharpe',
 'us_tics_download',
 'us_tics_short',
 'yfinfo']

In [79]:
#hdf5_rewrite('dat.h5', 'dat2.h5')